# Compagnon Immobilier - Économie & Démographie

## Extraction des données INSEE

---

Dans ce notebook, nous récupérons les données économiques et démographiques via l'API de l'INSEE (bibliothèque `pynsee`).

## 1. Configuration et Imports

In [1]:
# Imports standards
import os
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Imports pynsee
from pynsee.macrodata import get_dataset_list, get_series_list, get_series
from pynsee.localdata import get_local_data

# Configuration des chemins
PROJECT_ROOT = os.path.dirname(os.getcwd())  # Racine = Economie_Demographie
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')

# Créer le dossier si nécessaire
os.makedirs(DATA_DIR, exist_ok=True)

print(f"Dossier de données : {DATA_DIR}")
print(f"Date d'exécution : {datetime.now().strftime('%Y-%m-%d %H:%M')}")

Dossier de données : /home/tinkerbell/Desktop/DataScientest/Compagnon Immobilier - Datascientest/Economie_Demographie/data
Date d'exécution : 2025-12-24 12:49


## 2. Fonctionnement de pynsee

L'API pynsee propose deux modules principaux :

| Module | Fonction | Usage |
|--------|----------|-------|
| `macrodata` | `get_dataset_list()` | Liste des datasets macro disponibles |
| `macrodata` | `get_series_list(dataset)` | Séries temporelles d'un dataset |
| `macrodata` | `get_series(series_id)` | Données d'une série spécifique |
| `localdata` | `get_local_data()` | Données locales (départements, communes) |

In [2]:
# Explorer les datasets disponibles
datasets = get_dataset_list()
print(f"Nombre total de datasets : {len(datasets)}")
print(f"\nColonnes disponibles : {datasets.columns.tolist()}")
print("\n" + "="*80)
print("Premiers datasets :")
print("="*80)
datasets[['id', 'Name.fr', 'n_series']].head(10)

Nombre total de datasets : 226

Colonnes disponibles : ['id', 'Name.fr', 'Name.en', 'url', 'n_series']

Premiers datasets :


,id,Name.fr,n_series
0,BALANCE-PAIEMENTS,Balance des paiements,197
1,CHOMAGE-TRIM-NATIONAL,"Chômage, taux de chômage par sexe et âge (sens...",169
2,CLIMAT-AFFAIRES,Indicateurs synthétiques du climat des affaires,3
3,CNA-2010-CONSO-MEN,Consommation des ménages - Résultats par produ...,2247
4,CNA-2010-CONSO-SI,Dépenses de consommation finale par secteur in...,1391
5,CNA-2010-CPEB,Comptes de production et d’exploitation par br...,2739
6,CNA-2010-CSI,Comptes des secteurs institutionnels - Résulta...,1173
7,CNA-2010-DEP-APU,Dépenses des administrations publiques - Résul...,4400
8,CNA-2010-DETTE-APU,Dette et déficit des administrations publiques...,72
9,CNA-2010-EMPLOI,"Emploi intérieur, durée effective travaillée e...",702


### 2.2 Recherche de datasets pertinents

Nous recherchons les datasets en lien avec notre problématique immobilière :

In [3]:
# Fonction de recherche dans les datasets
def rechercher_datasets(mot_cle):
    """Recherche des datasets contenant un mot-clé."""
    mask = datasets['Name.fr'].str.lower().str.contains(mot_cle.lower(), na=False)
    resultats = datasets[mask][['id', 'Name.fr', 'n_series']]
    print(f"Recherche '{mot_cle}' : {len(resultats)} résultat(s)")
    return resultats

# Recherches pertinentes pour l'immobilier
print("=" * 80)
print("DATASETS PERTINENTS POUR L'IMMOBILIER")
print("=" * 80)

for terme in ['chomage', 'pib', 'prix', 'population', 'revenu', 'logement', 'inflation']:
    print(f"\n--- {terme.upper()} ---")
    display(rechercher_datasets(terme))

DATASETS PERTINENTS POUR L'IMMOBILIER

--- CHOMAGE ---
Recherche 'chomage' : 0 résultat(s)


,id,Name.fr,n_series



--- PIB ---
Recherche 'pib' : 3 résultat(s)


,id,Name.fr,n_series
14,CNA-2010-PIB,Produit intérieur brut (PIB) et ses composante...,34
29,CNA-2014-PIB,Produit intérieur brut (PIB) et ses composantes,25
40,CNA-2020-PIB,Produit intérieur brut (PIB) et ses composantes,21



--- PRIX ---
Recherche 'prix' : 29 résultat(s)


...

,id,Name.fr,n_series
15,CNA-2010-RDB,Revenu et pouvoir d’achat des ménages - Résult...,11
30,CNA-2014-RDB,Revenu et pouvoir d’achat des ménages,12
204,TCRED-SALAIRES-REVENUS-MEN,TCRED - Revenus disponibles des ménages,565
206,TCRED-SALAIRES-REVENUS-REV-SAL-SEXE-CS,TCRED - Revenu salarial annuel moyen selon la ...,819
207,TCRED-SALAIRES-REVENUS-STRUCTURE-RDB,TCRED - Structure du revenu disponible,565



--- LOGEMENT ---
Recherche 'logement' : 6 résultat(s)


,id,Name.fr,n_series
61,CONSTRUCTION-LOGEMENTS,Construction de logements,608
124,IPEA-2010,Indice des prix d’entretien-amélioration des l...,10
134,IPLA-IPLNA-2010,Indices des prix des logements neufs et indice...,112
135,IPLA-IPLNA-2015,Indices des prix des logements neufs et Indice...,142
167,PARC-LOGEMENTS,Estimations annuelles du parc de logements (EAPL),26
183,TCRED-CONDITIONSDEVIE-LOG-SOC,TCRED - Logements sociaux,468



--- INFLATION ---
Recherche 'inflation' : 0 résultat(s)


,id,Name.fr,n_series


---

## 3. Données macroéconomiques

### 3.1 PIB (Produit Intérieur Brut)

In [4]:
# Exploration du dataset PIB
dataset_pib = 'CNT-2020-PIB-EQB-RF'

# Lister les séries disponibles
series_pib = get_series_list(dataset_pib)
print(f"Dataset : {dataset_pib}")
print(f"Nombre de séries : {len(series_pib)}")
print(f"\nColonnes : {series_pib.columns.tolist()}")
series_pib.head()

Dataset : CNT-2020-PIB-EQB-RF
Nombre de séries : 68

Colonnes : ['DATASET', 'IDBANK', 'KEY', 'FREQ', 'INDICATEUR', 'SECT_INST', 'OPERATION', 'CNA_PRODUIT', 'NATURE', 'REF_AREA', 'VALORISATION', 'UNIT_MEASURE', 'CORRECTION', 'SERIE_ARRETEE', 'FREQ_label_fr', 'FREQ_label_en', 'INDICATEUR_label_fr', 'INDICATEUR_label_en', 'SECT_INST_label_fr', 'SECT_INST_label_en', 'OPERATION_label_fr', 'OPERATION_label_en', 'CNA_PRODUIT_label_fr', 'CNA_PRODUIT_label_en', 'NATURE_label_fr', 'NATURE_label_en', 'REF_AREA_label_fr', 'REF_AREA_label_en', 'VALORISATION_label_fr', 'VALORISATION_label_en', 'UNIT_MEASURE_label_fr', 'UNIT_MEASURE_label_en', 'CORRECTION_label_fr', 'CORRECTION_label_en', 'SERIE_ARRETEE_label_fr', 'SERIE_ARRETEE_label_en']


,DATASET,IDBANK,KEY,FREQ,INDICATEUR,SECT_INST,OPERATION,CNA_PRODUIT,NATURE,REF_AREA,...,REF_AREA_label_fr,REF_AREA_label_en,VALORISATION_label_fr,VALORISATION_label_en,UNIT_MEASURE_label_fr,UNIT_MEASURE_label_en,CORRECTION_label_fr,CORRECTION_label_en,SERIE_ARRETEE_label_fr,SERIE_ARRETEE_label_en
0,CNT-2020-PIB-EQB-RF,011794891,T.CNT-EQUILIBRE_PIB.S15.P51P.D-CNT.VALEUR_ABSO...,T,CNT-EQUILIBRE_PIB,S15,P51P,D-CNT,VALEUR_ABSOLUE,FE,...,France,France,Valeurs aux prix courants,Values at current prices,euros,euros,Corrigé des variations saisonnières et du nomb...,Seasonally and working-day adjusted,non,no
1,CNT-2020-PIB-EQB-RF,011794888,T.CNT-EQUILIBRE_PIB.S0.P51.D-CNT.VALEUR_ABSOLU...,T,CNT-EQUILIBRE_PIB,S0,P51,D-CNT,VALEUR_ABSOLUE,FE,...,France,France,Valeurs aux prix courants,Values at current prices,euros,euros,Corrigé des variations saisonnières et du nomb...,Seasonally and working-day adjusted,non,no
2,CNT-2020-PIB-EQB-RF,011794889,T.CNT-EQUILIBRE_PIB.S13.P51G.D-CNT.VALEUR_ABSO...,T,CNT-EQUILIBRE_PIB,S13,P51G,D-CNT,VALEUR_ABSOLUE,FE,...,France,France,Valeurs aux prix courants,Values at current prices,euros,euros,Corrigé des variations saisonnières et du nomb...,Seasonally and working-day adjusted,non,no
3,CNT-2020-PIB-EQB-RF,011794890,T.CNT-EQUILIBRE_PIB.S14.P51M.D-CNT.VALEUR_ABSO...,T,CNT-EQUILIBRE_PIB,S14,P51M,D-CNT,VALEUR_ABSOLUE,FE,...,France,France,Valeurs aux prix courants,Values at current prices,euros,euros,Corrigé des variations saisonnières et du nomb...,Seasonally and working-day adjusted,non,no
4,CNT-2020-PIB-EQB-RF,011794892,T.CNT-EQUILIBRE_PIB.S15.P3.SO.RATIO.FE.L.SO.CV...,T,CNT-EQUILIBRE_PIB,S15,P3,SO,RATIO,FE,...,France,France,Volumes aux prix de l'année précédente chaînés,Volumes chained at previous year prices,sans objet,not applicable,Corrigé des variations saisonnières et du nomb...,Seasonally and working-day adjusted,non,no


In [5]:
# Chargement ou extraction du PIB
fichier_pib = os.path.join(DATA_DIR, 'pib_france.csv')

if os.path.exists(fichier_pib):
    df_pib = pd.read_csv(fichier_pib)
    print(f"PIB chargé : {df_pib.shape[0]} lignes")
    print(f"Période : {df_pib['TIME_PERIOD'].min()} à {df_pib['TIME_PERIOD'].max()}")
else:
    idbanks_pib = series_pib['IDBANK'].tolist()
    df_pib = get_series(idbanks_pib)
    print(f"PIB extrait : {df_pib.shape}")

df_pib.head()

PIB chargé : 20435 lignes
Période : 1949-Q1 à 2025-Q3


,TIME_PERIOD,OBS_VALUE,OBS_STATUS,OBS_REV,OBS_QUAL,OBS_TYPE,IDBANK,FREQ,TITLE_FR,TITLE_EN,...,SERIE_ARRETEE_label_fr,SERIE_ARRETEE_label_en,OBS_STATUS_label_fr,OBS_STATUS_label_en,OBS_REV_label_fr,OBS_REV_label_en,OBS_TYPE_label_fr,OBS_TYPE_label_en,OBS_QUAL_label_fr,OBS_QUAL_label_en
0,1950-Q2,1.71,A,NaN,DEF,A,11794832,T,Demande intérieure finale - Contribution à l'é...,Final domestic demand - Contribution to GDP gr...,...,non,no,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value
1,1950-Q3,2.12,A,NaN,DEF,A,11794832,T,Demande intérieure finale - Contribution à l'é...,Final domestic demand - Contribution to GDP gr...,...,non,no,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value
2,1950-Q4,1.17,A,NaN,DEF,A,11794832,T,Demande intérieure finale - Contribution à l'é...,Final domestic demand - Contribution to GDP gr...,...,non,no,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value
3,1951-Q1,0.51,A,NaN,DEF,A,11794832,T,Demande intérieure finale - Contribution à l'é...,Final domestic demand - Contribution to GDP gr...,...,non,no,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value
4,1951-Q2,1.64,A,NaN,DEF,A,11794832,T,Demande intérieure finale - Contribution à l'é...,Final domestic demand - Contribution to GDP gr...,...,non,no,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value


In [6]:
# Sauvegarde des données PIB
fichier_pib = os.path.join(DATA_DIR, 'pib_france.csv')
df_pib.to_csv(fichier_pib, index=False)
print(f"Fichier sauvegardé : {fichier_pib}")
print(f"Taille : {os.path.getsize(fichier_pib) / 1024:.1f} KB")

Fichier sauvegardé : /home/tinkerbell/Desktop/DataScientest/Compagnon Immobilier - Datascientest/Economie_Demographie/data/pib_france.csv
Taille : 20239.9 KB


### 3.2 IPC (Indice des Prix à la Consommation)

In [7]:
# Dataset IPC
dataset_ipc = 'IPC-2015'

series_ipc = get_series_list(dataset_ipc)
print(f"Dataset : {dataset_ipc}")
print(f"Nombre de séries : {len(series_ipc)}")
series_ipc.head()

Dataset : IPC-2015
Nombre de séries : 4225


,DATASET,IDBANK,KEY,FREQ,INDICATEUR,FORME_VENTE,COICOP2016,PRIX_CONSO,NATURE,MENAGES_IPC,...,REF_AREA_label_fr,REF_AREA_label_en,UNIT_MEASURE_label_fr,UNIT_MEASURE_label_en,CORRECTION_label_fr,CORRECTION_label_en,BASIND_label_fr,BASIND_label_en,SERIE_ARRETEE_label_fr,SERIE_ARRETEE_label_en
0,IPC-2015,001765985,A.IPC.SO.045501.SO.POND.ENSEMBLE.FE.P10000.BRU...,A,IPC,SO,045501,SO,POND,ENSEMBLE,...,France,France,1/10.000,"1/10,000",Non corrigé,Uncorrected,2015,2015,non,no
1,IPC-2015,001765984,A.IPC.SO.04550.SO.POND.ENSEMBLE.FE.P10000.BRUT...,A,IPC,SO,04550,SO,POND,ENSEMBLE,...,France,France,1/10.000,"1/10,000",Non corrigé,Uncorrected,2015,2015,non,no
2,IPC-2015,001765983,A.IPC.SO.0455.SO.POND.ENSEMBLE.FE.P10000.BRUT....,A,IPC,SO,0455,SO,POND,ENSEMBLE,...,France,France,1/10.000,"1/10,000",Non corrigé,Uncorrected,2015,2015,non,no
3,IPC-2015,001765982,A.IPC.SO.045491.SO.POND.ENSEMBLE.FE.P10000.BRU...,A,IPC,SO,045491,SO,POND,ENSEMBLE,...,France,France,1/10.000,"1/10,000",Non corrigé,Uncorrected,2015,2015,non,no
4,IPC-2015,001765981,A.IPC.SO.04549.SO.POND.ENSEMBLE.FE.P10000.BRUT...,A,IPC,SO,04549,SO,POND,ENSEMBLE,...,France,France,1/10.000,"1/10,000",Non corrigé,Uncorrected,2015,2015,non,no


In [8]:
# Chargement ou extraction des données IPC
fichier_ipc = os.path.join(DATA_DIR, 'ipc_france.csv')

if os.path.exists(fichier_ipc):
    df_ipc = pd.read_csv(fichier_ipc)
    print(f"IPC chargé : {df_ipc.shape[0]} lignes")
    print(f"Période : {df_ipc['TIME_PERIOD'].min()} à {df_ipc['TIME_PERIOD'].max()}")
else:
    # Trop de séries (4225) - on filtre sur l'indice général
    series_filtrees = series_ipc[
        (series_ipc['FREQ'] == 'M') &
        (series_ipc['COICOP2016'].isin(['00', '0']))
    ]
    idbanks_ipc = series_filtrees['IDBANK'].tolist()
    df_ipc = get_series(idbanks_ipc)
    print(f"IPC extrait : {df_ipc.shape}")

df_ipc.head()

IPC chargé : 431 lignes
Période : 1990-01 à 2025-11


,TIME_PERIOD,OBS_VALUE,OBS_STATUS,OBS_QUAL,OBS_TYPE,IDBANK,FREQ,TITLE_FR,TITLE_EN,LAST_UPDATE,...,BASIND_label_fr,BASIND_label_en,SERIE_ARRETEE_label_fr,SERIE_ARRETEE_label_en,OBS_STATUS_label_fr,OBS_STATUS_label_en,OBS_TYPE_label_fr,OBS_TYPE_label_en,OBS_QUAL_label_fr,OBS_QUAL_label_en
0,1990-01,66.44,A,DEF,A,1763866,M,Indice des prix à la consommation - Base 2015 ...,Consumer price index - Base 2015 - All househo...,2025-12-12,...,2015,2015,non,no,Valeur normale,Normal value,Valeur normale,Normal value,Valeur définitive,Final value
1,1990-02,66.58,A,DEF,A,1763866,M,Indice des prix à la consommation - Base 2015 ...,Consumer price index - Base 2015 - All househo...,2025-12-12,...,2015,2015,non,no,Valeur normale,Normal value,Valeur normale,Normal value,Valeur définitive,Final value
2,1990-03,66.74,A,DEF,A,1763866,M,Indice des prix à la consommation - Base 2015 ...,Consumer price index - Base 2015 - All househo...,2025-12-12,...,2015,2015,non,no,Valeur normale,Normal value,Valeur normale,Normal value,Valeur définitive,Final value
3,1990-04,67.11,A,DEF,A,1763866,M,Indice des prix à la consommation - Base 2015 ...,Consumer price index - Base 2015 - All househo...,2025-12-12,...,2015,2015,non,no,Valeur normale,Normal value,Valeur normale,Normal value,Valeur définitive,Final value
4,1990-05,67.21,A,DEF,A,1763866,M,Indice des prix à la consommation - Base 2015 ...,Consumer price index - Base 2015 - All househo...,2025-12-12,...,2015,2015,non,no,Valeur normale,Normal value,Valeur normale,Normal value,Valeur définitive,Final value


In [9]:
# Sauvegarde IPC
fichier_ipc = os.path.join(DATA_DIR, 'ipc_france.csv')
df_ipc.to_csv(fichier_ipc, index=False)
print(f"Fichier sauvegardé : {fichier_ipc}")

Fichier sauvegardé : /home/tinkerbell/Desktop/DataScientest/Compagnon Immobilier - Datascientest/Economie_Demographie/data/ipc_france.csv


### 3.3 Prix des logements (Indices Notaires-INSEE)

In [10]:
# Recherche des datasets sur les prix des logements
datasets_logement = rechercher_datasets('logement')
display(datasets_logement)

Recherche 'logement' : 6 résultat(s)


,id,Name.fr,n_series
61,CONSTRUCTION-LOGEMENTS,Construction de logements,608
124,IPEA-2010,Indice des prix d’entretien-amélioration des l...,10
134,IPLA-IPLNA-2010,Indices des prix des logements neufs et indice...,112
135,IPLA-IPLNA-2015,Indices des prix des logements neufs et Indice...,142
167,PARC-LOGEMENTS,Estimations annuelles du parc de logements (EAPL),26
183,TCRED-CONDITIONSDEVIE-LOG-SOC,TCRED - Logements sociaux,468


In [11]:
# Indices des prix des logements (Notaires-INSEE)
dataset_prix_log = 'IPLA-IPLNA-2015'

series_prix = get_series_list(dataset_prix_log)
print(f"Séries disponibles : {len(series_prix)}")
series_prix.head(10)

Séries disponibles : 142


,DATASET,IDBANK,KEY,FREQ,INDICATEUR,NATURE,REF_AREA,UNIT_MEASURE,CORRECTION,BASIND,...,REF_AREA_label_fr,REF_AREA_label_en,UNIT_MEASURE_label_fr,UNIT_MEASURE_label_en,CORRECTION_label_fr,CORRECTION_label_en,BASIND_label_fr,BASIND_label_en,SERIE_ARRETEE_label_fr,SERIE_ARRETEE_label_en
0,IPLA-IPLNA-2015,010567050,T.IPLA_A.INDICE.D95.SO.BRUT.2015.FALSE,T,IPLA_A,INDICE,D95,SO,BRUT,2015,...,95 - Val-d'Oise,95 - Val-d'Oise,sans objet,not applicable,Non corrigé,Uncorrected,2015,2015,non,no
1,IPLA-IPLNA-2015,010567051,T.IPLA_A.INDICE.D95.SO.CVS.2015.FALSE,T,IPLA_A,INDICE,D95,SO,CVS,2015,...,95 - Val-d'Oise,95 - Val-d'Oise,sans objet,not applicable,Corrigé des variations saisonnières,Seasonal adjusted,2015,2015,non,no
2,IPLA-IPLNA-2015,010567052,T.IPLA_E.INDICE.D95.SO.BRUT.2015.FALSE,T,IPLA_E,INDICE,D95,SO,BRUT,2015,...,95 - Val-d'Oise,95 - Val-d'Oise,sans objet,not applicable,Non corrigé,Uncorrected,2015,2015,non,no
3,IPLA-IPLNA-2015,010567117,T.IPLA_A.INDICE.FR-D976.SO.CVS.2015.FALSE,T,IPLA_A,INDICE,FR-D976,SO,CVS,2015,...,France hors Mayotte,France excluding Mayotte,sans objet,not applicable,Corrigé des variations saisonnières,Seasonal adjusted,2015,2015,non,no
4,IPLA-IPLNA-2015,010567118,T.IPLA_E.INDICE.FR-D976.SO.BRUT.2015.FALSE,T,IPLA_E,INDICE,FR-D976,SO,BRUT,2015,...,France hors Mayotte,France excluding Mayotte,sans objet,not applicable,Non corrigé,Uncorrected,2015,2015,non,no
5,IPLA-IPLNA-2015,010567119,T.IPLA_E.INDICE.FR-D976.SO.CVS.2015.FALSE,T,IPLA_E,INDICE,FR-D976,SO,CVS,2015,...,France hors Mayotte,France excluding Mayotte,sans objet,not applicable,Corrigé des variations saisonnières,Seasonal adjusted,2015,2015,non,no
6,IPLA-IPLNA-2015,010567120,T.IPLA_M.INDICE.FR-D976.SO.BRUT.2015.FALSE,T,IPLA_M,INDICE,FR-D976,SO,BRUT,2015,...,France hors Mayotte,France excluding Mayotte,sans objet,not applicable,Non corrigé,Uncorrected,2015,2015,non,no
7,IPLA-IPLNA-2015,010567121,T.IPLA_M.INDICE.FR-D976.SO.CVS.2015.FALSE,T,IPLA_M,INDICE,FR-D976,SO,CVS,2015,...,France hors Mayotte,France excluding Mayotte,sans objet,not applicable,Corrigé des variations saisonnières,Seasonal adjusted,2015,2015,non,no
8,IPLA-IPLNA-2015,010567122,T.IPLA_A.INDICE.R32.SO.BRUT.2015.FALSE,T,IPLA_A,INDICE,R32,SO,BRUT,2015,...,Hauts-de-France,Hauts-de-France,sans objet,not applicable,Non corrigé,Uncorrected,2015,2015,non,no
9,IPLA-IPLNA-2015,010567123,T.IPLA_A.INDICE.R32.SO.CVS.2015.FALSE,T,IPLA_A,INDICE,R32,SO,CVS,2015,...,Hauts-de-France,Hauts-de-France,sans objet,not applicable,Corrigé des variations saisonnières,Seasonal adjusted,2015,2015,non,no


In [12]:
# Chargement ou extraction des prix des logements
fichier_prix = os.path.join(DATA_DIR, 'prix_logements_indices.csv')

if os.path.exists(fichier_prix):
    df_prix_log = pd.read_csv(fichier_prix)
    print(f"Prix logements chargé : {df_prix_log.shape[0]} lignes")
    print(f"Période : {df_prix_log['TIME_PERIOD'].min()} à {df_prix_log['TIME_PERIOD'].max()}")
else:
    idbanks_prix = series_prix['IDBANK'].tolist()
    df_prix_log = get_series(idbanks_prix)
    print(f"Prix logements extrait : {df_prix_log.shape}")

df_prix_log.head()

Prix logements chargé : 15296 lignes
Période : 1992-Q1 à 2025-Q3


,TIME_PERIOD,OBS_VALUE,OBS_STATUS,OBS_QUAL,OBS_TYPE,IDBANK,FREQ,TITLE_FR,TITLE_EN,LAST_UPDATE,...,OBS_STATUS_label_fr,OBS_STATUS_label_en,OBS_REV_label_fr,OBS_REV_label_en,OBS_CONF_label_fr,OBS_CONF_label_en,OBS_TYPE_label_fr,OBS_TYPE_label_en,OBS_QUAL_label_fr,OBS_QUAL_label_en
0,2000-Q1,47.7,A,DEF,A,10001868,T,Indice des prix des logements (neufs et ancien...,House price index (new and second-hand) – Raw ...,2025-12-19,...,Valeur normale,Normal value,NaN,NaN,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value
1,2000-Q2,49.1,A,DEF,A,10001868,T,Indice des prix des logements (neufs et ancien...,House price index (new and second-hand) – Raw ...,2025-12-19,...,Valeur normale,Normal value,NaN,NaN,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value
2,2000-Q3,50.8,A,DEF,A,10001868,T,Indice des prix des logements (neufs et ancien...,House price index (new and second-hand) – Raw ...,2025-12-19,...,Valeur normale,Normal value,NaN,NaN,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value
3,2000-Q4,50.6,A,DEF,A,10001868,T,Indice des prix des logements (neufs et ancien...,House price index (new and second-hand) – Raw ...,2025-12-19,...,Valeur normale,Normal value,NaN,NaN,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value
4,2001-Q1,51.2,A,DEF,A,10001868,T,Indice des prix des logements (neufs et ancien...,House price index (new and second-hand) – Raw ...,2025-12-19,...,Valeur normale,Normal value,NaN,NaN,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value


In [13]:
# Sauvegarde
fichier_prix = os.path.join(DATA_DIR, 'prix_logements_indices.csv')
df_prix_log.to_csv(fichier_prix, index=False)
print(f"Fichier sauvegardé : {fichier_prix}")

Fichier sauvegardé : /home/tinkerbell/Desktop/DataScientest/Compagnon Immobilier - Datascientest/Economie_Demographie/data/prix_logements_indices.csv


---

## 4. Données de chômage

### 4.1 Chômage national

In [14]:
# Dataset chômage
dataset_chomage = 'CHOMAGE-TRIM-NATIONAL'

series_chomage = get_series_list(dataset_chomage)
print(f"Dataset : {dataset_chomage}")
print(f"Nombre de séries : {len(series_chomage)}")
series_chomage.head()

Dataset : CHOMAGE-TRIM-NATIONAL
Nombre de séries : 169


,DATASET,IDBANK,KEY,FREQ,INDICATEUR,NATURE,REF_AREA,SEXE,AGE,UNIT_MEASURE,...,SEXE_label_fr,SEXE_label_en,AGE_label_fr,AGE_label_en,UNIT_MEASURE_label_fr,UNIT_MEASURE_label_en,CORRECTION_label_fr,CORRECTION_label_en,SERIE_ARRETEE_label_fr,SERIE_ARRETEE_label_en
0,CHOMAGE-TRIM-NATIONAL,001688537,T.CTTXC.TAUX.FR-D976.0.00-24.POURCENT.CVS.FALSE,T,CTTXC,TAUX,FR-D976,0,00-24,POURCENT,...,Ensemble,All,Moins de 25 ans,Below 25 years old,%,%,Corrigé des variations saisonnières,Seasonal adjusted,non,no
1,CHOMAGE-TRIM-NATIONAL,001688538,T.CTTXC.TAUX.FM.2.25-49.POURCENT.CVS.FALSE,T,CTTXC,TAUX,FM,2,25-49,POURCENT,...,Femmes,Women,De 25 à 49 ans,From 25 to 49 years old,%,%,Corrigé des variations saisonnières,Seasonal adjusted,non,no
2,CHOMAGE-TRIM-NATIONAL,001688531,T.CTTXC.TAUX.FR-D976.0.50-.POURCENT.CVS.FALSE,T,CTTXC,TAUX,FR-D976,0,50-,POURCENT,...,Ensemble,All,50 ans et plus,50 years old and over,%,%,Corrigé des variations saisonnières,Seasonal adjusted,non,no
3,CHOMAGE-TRIM-NATIONAL,001688536,T.CTTXC.TAUX.FM.0.00-24.POURCENT.CVS.FALSE,T,CTTXC,TAUX,FM,0,00-24,POURCENT,...,Ensemble,All,Moins de 25 ans,Below 25 years old,%,%,Corrigé des variations saisonnières,Seasonal adjusted,non,no
4,CHOMAGE-TRIM-NATIONAL,001688535,T.CTTXC.TAUX.FR-D976.1.00-.POURCENT.CVS.FALSE,T,CTTXC,TAUX,FR-D976,1,00-,POURCENT,...,Hommes,Men,Ensemble,All ages,%,%,Corrigé des variations saisonnières,Seasonal adjusted,non,no


In [15]:
# Chargement ou extraction du chômage national
fichier_chomage = os.path.join(DATA_DIR, 'chomage_series.csv')

if os.path.exists(fichier_chomage):
    df_chomage = pd.read_csv(fichier_chomage)
    print(f"Chômage national chargé : {df_chomage.shape[0]} lignes")
    print(f"Période : {df_chomage['TIME_PERIOD'].min()} à {df_chomage['TIME_PERIOD'].max()}")
else:
    idbanks_chomage = series_chomage['IDBANK'].tolist()
    df_chomage = get_series(idbanks_chomage)
    print(f"Chômage national extrait : {df_chomage.shape}")

df_chomage.head()

Chômage national chargé : 19130 lignes
Période : 1975-Q1 à 2025-Q3


,TIME_PERIOD,OBS_VALUE,OBS_STATUS,OBS_QUAL,OBS_TYPE,IDBANK,FREQ,TITLE_FR,TITLE_EN,LAST_UPDATE,...,SERIE_ARRETEE_label_fr,SERIE_ARRETEE_label_en,OBS_STATUS_label_fr,OBS_STATUS_label_en,OBS_REV_label_fr,OBS_REV_label_en,OBS_TYPE_label_fr,OBS_TYPE_label_en,OBS_QUAL_label_fr,OBS_QUAL_label_en
0,1975-Q1,644.0,A,DEF,A,1688358,T,Chômeurs au sens du BIT - Ensemble (en millier...,ILO unemployed - All (in thousands) - Metropol...,2019-08-14,...,oui,yes,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value
1,1975-Q2,718.0,A,DEF,A,1688358,T,Chômeurs au sens du BIT - Ensemble (en millier...,ILO unemployed - All (in thousands) - Metropol...,2019-08-14,...,oui,yes,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value
2,1975-Q3,790.0,A,DEF,A,1688358,T,Chômeurs au sens du BIT - Ensemble (en millier...,ILO unemployed - All (in thousands) - Metropol...,2019-08-14,...,oui,yes,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value
3,1975-Q4,812.0,A,DEF,A,1688358,T,Chômeurs au sens du BIT - Ensemble (en millier...,ILO unemployed - All (in thousands) - Metropol...,2019-08-14,...,oui,yes,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value
4,1976-Q1,822.0,A,DEF,A,1688358,T,Chômeurs au sens du BIT - Ensemble (en millier...,ILO unemployed - All (in thousands) - Metropol...,2019-08-14,...,oui,yes,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value


In [16]:
# Sauvegarde
fichier_chomage = os.path.join(DATA_DIR, 'chomage_series.csv')
df_chomage.to_csv(fichier_chomage, index=False)
print(f"Fichier sauvegardé : {fichier_chomage}")

Fichier sauvegardé : /home/tinkerbell/Desktop/DataScientest/Compagnon Immobilier - Datascientest/Economie_Demographie/data/chomage_series.csv


### 4.2 Chômage par département (taux localisé)

In [17]:
# Taux de chômage localisé (par département/région)
dataset_chomage_loc = 'TAUX-CHOMAGE'

series_chomage_loc = get_series_list(dataset_chomage_loc)
print(f"Dataset : {dataset_chomage_loc}")
print(f"Nombre de séries : {len(series_chomage_loc)}")
series_chomage_loc.head(10)

Dataset : TAUX-CHOMAGE
Nombre de séries : 115


,DATASET,IDBANK,KEY,FREQ,INDICATEUR,NATURE,REF_AREA,UNIT_MEASURE,CORRECTION,FREQ_label_fr,...,INDICATEUR_label_fr,INDICATEUR_label_en,NATURE_label_fr,NATURE_label_en,REF_AREA_label_fr,REF_AREA_label_en,UNIT_MEASURE_label_fr,UNIT_MEASURE_label_en,CORRECTION_label_fr,CORRECTION_label_en
0,TAUX-CHOMAGE,001515957,T.TAUX_CHOMAGE_LOCALISE.PROPORTION.D93.POURCEN...,T,TAUX_CHOMAGE_LOCALISE,PROPORTION,D93,POURCENT,CVS,Trimestrielle,...,Taux de chômage localisé,Unemployment rates localized,Proportion,Proportion,93 - Seine-Saint-Denis,93 - Seine-Saint-Denis,%,%,Corrigé des variations saisonnières,Seasonal adjusted
1,TAUX-CHOMAGE,001515956,T.TAUX_CHOMAGE_LOCALISE.PROPORTION.D92.POURCEN...,T,TAUX_CHOMAGE_LOCALISE,PROPORTION,D92,POURCENT,CVS,Trimestrielle,...,Taux de chômage localisé,Unemployment rates localized,Proportion,Proportion,92 - Hauts-de-Seine,92 - Hauts-de-Seine,%,%,Corrigé des variations saisonnières,Seasonal adjusted
2,TAUX-CHOMAGE,001515955,T.TAUX_CHOMAGE_LOCALISE.PROPORTION.D91.POURCEN...,T,TAUX_CHOMAGE_LOCALISE,PROPORTION,D91,POURCENT,CVS,Trimestrielle,...,Taux de chômage localisé,Unemployment rates localized,Proportion,Proportion,91 - Essonne,91 - Essonne,%,%,Corrigé des variations saisonnières,Seasonal adjusted
3,TAUX-CHOMAGE,001515954,T.TAUX_CHOMAGE_LOCALISE.PROPORTION.D90.POURCEN...,T,TAUX_CHOMAGE_LOCALISE,PROPORTION,D90,POURCENT,CVS,Trimestrielle,...,Taux de chômage localisé,Unemployment rates localized,Proportion,Proportion,90 - Territoire de Belfort,90 - Territoire de Belfort,%,%,Corrigé des variations saisonnières,Seasonal adjusted
4,TAUX-CHOMAGE,001515953,T.TAUX_CHOMAGE_LOCALISE.PROPORTION.D89.POURCEN...,T,TAUX_CHOMAGE_LOCALISE,PROPORTION,D89,POURCENT,CVS,Trimestrielle,...,Taux de chômage localisé,Unemployment rates localized,Proportion,Proportion,89 - Yonne,89 - Yonne,%,%,Corrigé des variations saisonnières,Seasonal adjusted
5,TAUX-CHOMAGE,001515949,T.TAUX_CHOMAGE_LOCALISE.PROPORTION.D85.POURCEN...,T,TAUX_CHOMAGE_LOCALISE,PROPORTION,D85,POURCENT,CVS,Trimestrielle,...,Taux de chômage localisé,Unemployment rates localized,Proportion,Proportion,85 - Vendée,85 - Vendée,%,%,Corrigé des variations saisonnières,Seasonal adjusted
6,TAUX-CHOMAGE,001515951,T.TAUX_CHOMAGE_LOCALISE.PROPORTION.D87.POURCEN...,T,TAUX_CHOMAGE_LOCALISE,PROPORTION,D87,POURCENT,CVS,Trimestrielle,...,Taux de chômage localisé,Unemployment rates localized,Proportion,Proportion,87 - Haute-Vienne,87 - Haute-Vienne,%,%,Corrigé des variations saisonnières,Seasonal adjusted
7,TAUX-CHOMAGE,001515950,T.TAUX_CHOMAGE_LOCALISE.PROPORTION.D86.POURCEN...,T,TAUX_CHOMAGE_LOCALISE,PROPORTION,D86,POURCENT,CVS,Trimestrielle,...,Taux de chômage localisé,Unemployment rates localized,Proportion,Proportion,86 - Vienne,86 - Vienne,%,%,Corrigé des variations saisonnières,Seasonal adjusted
8,TAUX-CHOMAGE,001515948,T.TAUX_CHOMAGE_LOCALISE.PROPORTION.D84.POURCEN...,T,TAUX_CHOMAGE_LOCALISE,PROPORTION,D84,POURCENT,CVS,Trimestrielle,...,Taux de chômage localisé,Unemployment rates localized,Proportion,Proportion,84 - Vaucluse,84 - Vaucluse,%,%,Corrigé des variations saisonnières,Seasonal adjusted
9,TAUX-CHOMAGE,001515947,T.TAUX_CHOMAGE_LOCALISE.PROPORTION.D82.POURCEN...,T,TAUX_CHOMAGE_LOCALISE,PROPORTION,D82,POURCENT,CVS,Trimestrielle,...,Taux de chômage localisé,Unemployment rates localized,Proportion,Proportion,82 - Tarn-et-Garonne,82 - Tarn-et-Garonne,%,%,Corrigé des variations saisonnières,Seasonal adjusted


In [18]:
# Chargement ou extraction du chômage localisé
fichier_chomage_dept = os.path.join(DATA_DIR, 'chomage_departements_complet.csv')

if os.path.exists(fichier_chomage_dept):
    df_chomage_dept = pd.read_csv(fichier_chomage_dept)
    print(f"Chômage localisé chargé : {df_chomage_dept.shape[0]} lignes")
    print(f"Période : {df_chomage_dept['TIME_PERIOD'].min()} à {df_chomage_dept['TIME_PERIOD'].max()}")
else:
    idbanks_chomage_loc = series_chomage_loc['IDBANK'].tolist()
    df_chomage_dept = get_series(idbanks_chomage_loc)
    print(f"Chômage localisé extrait : {df_chomage_dept.shape}")

df_chomage_dept.head()

Chômage localisé chargé : 16988 lignes
Période : 1982-Q1 à 2025-Q3


,TIME_PERIOD,OBS_VALUE,OBS_STATUS,OBS_QUAL,OBS_TYPE,IDBANK,FREQ,TITLE_FR,TITLE_EN,LAST_UPDATE,...,OBS_STATUS_label_fr,OBS_STATUS_label_en,OBS_REV_label_fr,OBS_REV_label_en,OBS_TYPE_label_fr,OBS_TYPE_label_en,OBS_QUAL_label_fr,OBS_QUAL_label_en,DEPT_CODE,REGION
0,1982-Q1,5.2,A,DEF,A,1515862,T,Taux de chômage localisé par département - Ave...,Unemployment rates localized by department - A...,2025-12-19,...,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value,12,Occitanie
1,1982-Q2,5.3,A,DEF,A,1515862,T,Taux de chômage localisé par département - Ave...,Unemployment rates localized by department - A...,2025-12-19,...,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value,12,Occitanie
2,1982-Q3,5.5,A,DEF,A,1515862,T,Taux de chômage localisé par département - Ave...,Unemployment rates localized by department - A...,2025-12-19,...,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value,12,Occitanie
3,1982-Q4,5.5,A,DEF,A,1515862,T,Taux de chômage localisé par département - Ave...,Unemployment rates localized by department - A...,2025-12-19,...,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value,12,Occitanie
4,1983-Q1,5.6,A,DEF,A,1515862,T,Taux de chômage localisé par département - Ave...,Unemployment rates localized by department - A...,2025-12-19,...,Valeur normale,Normal value,NaN,NaN,Valeur normale,Normal value,Valeur définitive,Final value,12,Occitanie


In [19]:
# Sauvegarde chômage départements
fichier_chomage_dept = os.path.join(DATA_DIR, 'chomage_departements_complet.csv')
df_chomage_dept.to_csv(fichier_chomage_dept, index=False)
print(f"Fichier sauvegardé : {fichier_chomage_dept}")
print(f"Taille : {os.path.getsize(fichier_chomage_dept) / 1024:.1f} KB")

Fichier sauvegardé : /home/tinkerbell/Desktop/DataScientest/Compagnon Immobilier - Datascientest/Economie_Demographie/data/chomage_departements_complet.csv
Taille : 9874.0 KB


---

## 5. Données locales (par département)

### 5.1 Population

In [20]:
# Liste des codes départements français
codes_departements = [f"{i:02d}" for i in range(1, 96) if i != 20]  # 01 à 95 sauf 20
codes_departements.extend(['2A', '2B'])  # Corse
codes_departements.extend(['971', '972', '973', '974', '976'])  # DOM

print(f"Nombre de départements : {len(codes_departements)}")
print(f"Premiers codes : {codes_departements[:10]}")

Nombre de départements : 101
Premiers codes : ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']


In [21]:
# Chargement des données de population
fichier_pop = os.path.join(DATA_DIR, 'population_departements.csv')

if os.path.exists(fichier_pop):
    df_pop = pd.read_csv(fichier_pop)
    print(f"Population chargé : {df_pop.shape[0]} lignes")
    display(df_pop.head())
else:
    print("Extraction population via get_local_data...")
    try:
        df_pop = get_local_data(
            variables='SEXE',
            dataset_version='GEO2024RP2021',
            nivgeo='DEP',
            geocodes=codes_departements[:5]
        )
        print(f"Population extrait : {df_pop.shape}")
    except Exception as e:
        print(f"Erreur : {e}")

Population chargé : 2003 lignes


,CODEGEO,NIVGEO,UNIT_label_fr,UNIT,STOCD,OBS_VALUE,STOCD_label,DATASET_VERSION,DATASET_NAME,DATA_DATE
0,01,DEP,Nombre de logements,NBLOG,ENS,274538.375368,Statut d'occupation du logement,GEO2021RP2018,Recensement de la population,2018.0
1,01,DEP,Nombre de logements,NBLOG,10,171307.121971,Statut d'occupation du logement,GEO2021RP2018,Recensement de la population,2018.0
2,01,DEP,Nombre de logements,NBLOG,21,54028.486208,Statut d'occupation du logement,GEO2021RP2018,Recensement de la population,2018.0
3,01,DEP,Nombre de logements,NBLOG,22,39004.585669,Statut d'occupation du logement,GEO2021RP2018,Recensement de la population,2018.0
4,01,DEP,Nombre de logements,NBLOG,23,5205.883730,Statut d'occupation du logement,GEO2021RP2018,Recensement de la population,2018.0


### 5.2 Revenus (FILOSOFI)

In [22]:
# Les données de revenus sont souvent dans des datasets spécifiques
# FILOSOFI (Fichier Localisé Social et Fiscal)

datasets_revenus = rechercher_datasets('revenu')
display(datasets_revenus)

Recherche 'revenu' : 5 résultat(s)


,id,Name.fr,n_series
15,CNA-2010-RDB,Revenu et pouvoir d’achat des ménages - Résult...,11
30,CNA-2014-RDB,Revenu et pouvoir d’achat des ménages,12
204,TCRED-SALAIRES-REVENUS-MEN,TCRED - Revenus disponibles des ménages,565
206,TCRED-SALAIRES-REVENUS-REV-SAL-SEXE-CS,TCRED - Revenu salarial annuel moyen selon la ...,819
207,TCRED-SALAIRES-REVENUS-STRUCTURE-RDB,TCRED - Structure du revenu disponible,565


In [23]:
# Chargement des données de revenus
fichier_revenus = os.path.join(DATA_DIR, 'revenus_departements.csv')

if os.path.exists(fichier_revenus):
    df_revenus = pd.read_csv(fichier_revenus)
    print(f"Revenus chargé : {df_revenus.shape[0]} lignes")
    display(df_revenus.head())
else:
    print("Fichier revenus_departements.csv non trouvé")
    print("Extraction via get_local_data (peut être long)...")
    try:
        df_revenus = get_local_data(
            variables='REVENU',
            dataset_version='GEO2024FILOSOFI2021',
            nivgeo='DEP',
            geocodes=codes_departements
        )
        print(f"Revenus extrait : {df_revenus.shape}")
    except Exception as e:
        print(f"Erreur : {e}")

Revenus chargé : 1770 lignes


,CODEGEO,NIVGEO,UNIT_label_fr,UNIT,INDICS_FILO_DISP,OBS_VALUE,INDICS_FILO_DISP_label,DATASET_VERSION,DATASET_NAME,DATA_DATE,INDICS_FILO_DISP_DET,INDICS_FILO_DISP_DET_label
0,01,DEP,Nombre de ménages fiscaux,NBMEN,1.0,258018.0,Indicateur de revenus disponibles,GEO2021FILO2018,Fichier localisé social et fiscal,2018.0,NaN,NaN
1,01,DEP,Nombre de personnes fiscales,NBPERS,1.0,618263.0,Indicateur de revenus disponibles,GEO2021FILO2018,Fichier localisé social et fiscal,2018.0,NaN,NaN
2,01,DEP,Médiane,MEDIANE,1.0,23420.0,Indicateur de revenus disponibles,GEO2021FILO2018,Fichier localisé social et fiscal,2018.0,NaN,NaN
3,02,DEP,Nombre de ménages fiscaux,NBMEN,1.0,221370.0,Indicateur de revenus disponibles,GEO2021FILO2018,Fichier localisé social et fiscal,2018.0,NaN,NaN
4,02,DEP,Nombre de personnes fiscales,NBPERS,1.0,512838.0,Indicateur de revenus disponibles,GEO2021FILO2018,Fichier localisé social et fiscal,2018.0,NaN,NaN


---

## 6. Récapitulatif des fichiers extraits

In [24]:
# Récapitulatif de tous les fichiers dans le dossier data
print("=" * 80)
print("FICHIERS DE DONNÉES DISPONIBLES")
print("=" * 80)

fichiers_csv = [f for f in os.listdir(DATA_DIR) if f.endswith('.csv')]
fichiers_csv.sort()

total_size = 0
for f in fichiers_csv:
    chemin = os.path.join(DATA_DIR, f)
    size = os.path.getsize(chemin) / 1024
    total_size += size
    
    # Charger pour avoir le nombre de lignes
    try:
        df = pd.read_csv(chemin, sep=';')
    except:
        df = pd.read_csv(chemin)
    print(f"{f:45} | {size:8.1f} KB | {len(df):7} lignes | {len(df.columns):2} cols")

print("=" * 80)
print(f"Total : {len(fichiers_csv)} fichiers | {total_size/1024:.1f} MB")

FICHIERS DE DONNÉES DISPONIBLES


ParserError: Error tokenizing data. C error: Expected 2 fields in line 4, saw 231


### 6.1 Construction de logements (permis et mises en chantier)

In [ ]:
# Construction de logements - permis de construire et mises en chantier
fichier_constr = os.path.join(DATA_DIR, 'construction_logements.csv')

if os.path.exists(fichier_constr):
    df_constr = pd.read_csv(fichier_constr)
    print(f"Construction logements chargé : {df_constr.shape[0]} lignes")
    print(f"Période : {df_constr['TIME_PERIOD'].min()} à {df_constr['TIME_PERIOD'].max()}")
else:
    series_constr = get_series_list('CONSTRUCTION-LOGEMENTS')
    # Filtrer sur France et régions, ensemble des logements
    series_filtre = series_constr[
        (series_constr['REF_AREA'].str.startswith('FM') | series_constr['REF_AREA'].str.startswith('R')) &
        (series_constr['LOGEMENT'] == '0')
    ]
    idbanks = series_filtre['IDBANK'].tolist()
    df_constr = get_series(idbanks)
    df_constr.to_csv(fichier_constr, index=False)
    print(f"Construction logements extrait : {df_constr.shape[0]} lignes")

df_constr.head()

### 6.2 Parc de logements (stock annuel)

In [ ]:
# Parc de logements - estimations annuelles (EAPL)
fichier_parc = os.path.join(DATA_DIR, 'parc_logements.csv')

if os.path.exists(fichier_parc):
    df_parc = pd.read_csv(fichier_parc)
    print(f"Parc logements chargé : {df_parc.shape[0]} lignes")
    print(f"Période : {df_parc['TIME_PERIOD'].min()} à {df_parc['TIME_PERIOD'].max()}")
else:
    series_parc = get_series_list('PARC-LOGEMENTS')
    idbanks = series_parc['IDBANK'].tolist()
    df_parc = get_series(idbanks)
    df_parc.to_csv(fichier_parc, index=False)
    print(f"Parc logements extrait : {df_parc.shape[0]} lignes")

df_parc.head()

---

## 7. Limites des données extraites

Certaines données présentent des limites qu'il faut prendre en compte pour l'analyse :

| Donnée | Limite | Solution appliquée |
|--------|--------|-------------------|
| **Revenus ménages (API)** | 2012-2015 uniquement | FILOSOFI 2021 téléchargé manuellement |
| **Pauvreté (API)** | 2012-2015 uniquement | FILOSOFI 2021 téléchargé manuellement |
| **Prix logements** | 32 zones seulement | Données DVF gérées par un collègue |
| **FILOSOFI via API** | 2018 max | Téléchargement direct depuis insee.fr |

### Données complémentaires intégrées

- **FILOSOFI 2021** : Revenus et pauvreté par commune (téléchargé depuis https://www.insee.fr/fr/statistiques/6692392)
- **Construction logements** : Permis et mises en chantier (API INSEE)
- **Parc logements** : Stock annuel par catégorie (API INSEE)

### Note sur les prix immobiliers

Les données DVF (Demandes de Valeurs Foncières) au niveau communal sont gérées par un autre membre de l'équipe via https://app.dvf.etalab.gouv.fr/

---

## 8. Fonction utilitaire

Fonction réutilisable pour extraire et sauvegarder un dataset :

In [ ]:
def extraire_et_sauvegarder(dataset_id, nom_fichier, dossier=DATA_DIR):
    """
    Extrait un dataset INSEE et le sauvegarde en CSV.
    
    Paramètres:
        dataset_id (str): Identifiant du dataset INSEE
        nom_fichier (str): Nom du fichier de sortie (avec .csv)
        dossier (str): Dossier de destination
    
    Retourne:
        DataFrame ou None en cas d'erreur
    """
    try:
        print(f"Extraction de {dataset_id}...")
        
        # 1. Récupérer la liste des séries et leurs IDBANK
        series_list = get_series_list(dataset_id)
        idbanks = series_list['IDBANK'].tolist()
        print(f"  -> {len(idbanks)} séries trouvées")
        
        # 2. Extraire les données
        df = get_series(idbanks)
        
        # 3. Sauvegarde
        chemin = os.path.join(dossier, nom_fichier)
        df.to_csv(chemin, index=False)
        
        print(f"  -> {len(df)} lignes extraites")
        print(f"  -> Sauvegardé : {nom_fichier}")
        
        return df
        
    except Exception as e:
        print(f"  -> Erreur : {e}")
        return None

In [ ]:
# Exemple d'utilisation
# extraire_et_sauvegarder('CNT-2020-PIB-EQB-RF', 'pib_france_test.csv')

---

## Bilan

**Sources de données extraites :**

| Source | Fichiers | Contenu |
|--------|----------|---------|
| API INSEE (pynsee) | 14 fichiers | PIB, IPC, chômage, population, emploi, prix logements |
| Téléchargement INSEE | 6 fichiers | FILOSOFI 2021 (revenus et pauvreté par commune) |
| API INSEE | 2 fichiers | Construction et parc de logements |

Ces données devront être nettoyées et harmonisées dans le notebook suivant (`03_nettoyage_donnees.ipynb`).